In [1]:
import os

In [2]:
%pwd

'c:\\Users\\diban\\Documents\\End-end ML project with MLFlow\\End-end-Projects-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir :Path
    source_URL :str
    local_data_file :Path
    unzip_dir :Path
    
    #This is like creating a container and the frozen =True is to make the container immutable

In [5]:
from src.ML_Project.constants import *
from src.ML_Project.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH
                     ,params_filepath = PARAMS_FILE_PATH
                         ,schema_filepath = SCHEMA_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root]) #creating the artifact_root path
        


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion #this is reading the yaml file which points to config file and open the data_ingestion with conffig box
        
        create_directories([config.root_dir]) #using this function from common ro create dictionary accessed from the #self.config.data_ingestion above
        
        
        data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir
        )#loading the container
        
        return data_ingestion_config
    
    
        







In [7]:
#updating the components
import os
import urllib.request as request
import zipfile
from src.ML_Project import Logger
from src.ML_Project.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        

    def download_file(self):
        """downloading data and return data size if data already exist
        """
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url =self.config.source_URL,
                filename = self.config.local_data_file
            )
            Logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            Logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            

    def extract_zip(self):
        """
        extract zip file into the data directory
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)



In [9]:
#updating my pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()

except Exception as e:
    raise e


[2025-01-16 14:55:54,992:INFO:yaml file :config\config.yaml loaded successfully]
[2025-01-16 14:55:54,995:INFO:yaml file :params.yaml loaded successfully]
[2025-01-16 14:55:54,995:INFO:yaml file :schema.yaml loaded successfully]
[2025-01-16 14:55:54,995:INFO:created directory at: artifacts]
[2025-01-16 14:55:54,995:INFO:created directory at: artifacts/data_ingestion]


TypeError: unsupported operand type(s) for /: 'WindowsPath' and 'int'